# From MJy/sr to erg s-1 cm-2 A-1

This macro will take SKIRT's FITS output and convert it to another unit i.e. $erg$ $s^{-1}$ $cm^{-2}$ $\mathring A^{-1}$

In [154]:
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
import numpy as np

In [132]:
#Dfining some constants
speed_of_ligth = 2.998e18 #Ang/s
arcsec_to_rad = np.pi / 180 / 3600

In [133]:
#Loading file
subhalo_path = '/Users/.../subhalos/' #my directory of SKIRT outputs
subhalo_id = 509091 #from TNG50 at z=0
instrument_name = 'model4_resolved_muse' #name given for the 3d cube output
extension = '_total-copy.fits' #SKIRT's extension for a cube instrument

path = subhalo_path + str(subhalo_id)+ '/' + instrument_name + extension

In [147]:
#Opening FITS file
hdulist = fits.open(path,mode='update')
prihdr = hdulist[0].header

In [148]:
hdulist.info()

Filename: /Users/.../subhalos/509091/model4_resolved_muse_total-copy.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      25   (333, 323, 3681)   float32   
  1  Z-axis coordinate values    1 TableHDU        13   3681R x 1C   [E16.9]   


In [149]:
prihdr

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    3 / number of data axes                            
NAXIS1  =                  333 / length of data axis 1                          
NAXIS2  =                  323 / length of data axis 2                          
NAXIS3  =                 3681 / length of data axis 3                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BSCALE  =                    1 / Array value scale                              
BZERO   =                    0 / Array value offset                             
DATE    = '2022-09-21T15:36:00' / Date and time of creation (UTC)               
ORIGIN  = 'SKIRT simulation'

In [150]:
#Lets modify some keywords
prihdr['BUNIT'] = ('erg s-1 cm-2 A-1','Physical unit of the array values')
prihdr['history'] = 'I updated this file on 27/sept/22'
prihdr['CUNIT3'] = ('angstrom','Physical units of the Z-axis')

In [151]:
prihdr

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    3 / number of data axes                            
NAXIS1  =                  333 / length of data axis 1                          
NAXIS2  =                  323 / length of data axis 2                          
NAXIS3  =                 3681 / length of data axis 3                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BSCALE  =                    1 / Array value scale                              
BZERO   =                    0 / Array value offset                             
DATE    = '2022-09-21T15:36:00' / Date and time of creation (UTC)               
ORIGIN  = 'SKIRT simulation'

In [139]:
#Saving some variables
pix_size_x = hdulist[0].header['CDELT1']
pix_size_y = hdulist[0].header['CDELT2']
x = hdulist[0].data.shape[2] #x dimension
y = hdulist[0].data.shape[1] #y dimension
z = hdulist[0].data.shape[0] #number of channels (wavelengths in microns)

#arcsec to radians 
radx = pix_size_x * arcsec_to_rad
rady = pix_size_y * arcsec_to_rad

In [140]:
#Getting all wavelengths (channels) and converting them from um to ang
scidata = hdulist[1].data
scidata_float = scidata.astype(float)
wave = scidata_float*1e4 #in Angstroms

In [141]:
from datetime import datetime
startTime = datetime.now()
print (startTime)

#Converting from MJy/sr to erg s-1 cm-2 A-1
conv_1 = 10e-23 * 10e6 * speed_of_ligth * radx * rady
for _z in range (z):
    hdulist[0].data[_z] = hdulist[0].data[_z] * conv_1 / ((wave[_z])**2)

print('Execution time:', datetime.now() - startTime)


2022-09-27 15:41:52.397878
Execution time: 0:00:06.620104


In [152]:
hdulist.flush() # changes are written back to path

In [153]:
hdulist.close()